## MultipleNegativesRankingLoss retail data

In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sentence_transformers import losses, util
from sentence_transformers import LoggingHandler, SentenceTransformer, evaluation
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import csv
import os
from zipfile import ZipFile
import random

In [2]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)
#### /print debug information to stdout

In [3]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch_device = 'cpu'

model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1',
                           device=torch_device)
num_epochs = 5
train_batch_size = 64

2022-11-16 09:35:50 - Load pretrained SentenceTransformer: multi-qa-MiniLM-L6-cos-v1


In [4]:
dataset_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                            'data/retail_train/')

model_save_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                               'data/output/training_retail_MultipleNegativesRankingLoss-')\
+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


os.makedirs(model_save_path, exist_ok=True)

In [6]:
######### Read train data  ##########
# Read train data
train_samples = []
with open(os.path.join(dataset_path, "classification/train_pairs.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row['is_duplicate'] == '1':
            train_samples.append(InputExample(texts=[row['product1'], 
                                                     row['product2']], 
                                              label=1))
            train_samples.append(InputExample(texts=[row['product2'], 
                                                     row['product1']], 
                                              label=1)) #if A is a duplicate of B, then B is a duplicate of A

# After reading the train_samples, we create a DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model)

In [13]:
################### Development  Evaluators ##################
# We add 3 evaluators, that evaluate the model on Duplicate Products pair classification,
# Duplicate Products Mining, and Duplicate Products Information Retrieval
evaluators = []

###### Classification ######
# Given (product1, product2), is this a duplicate or not?
# The evaluator will compute the embeddings for both products and then compute
# a cosine similarity. If the similarity is above a threshold, we have a duplicate.
dev_sentences1 = []
dev_sentences2 = []
dev_labels = []
with open(os.path.join(dataset_path, "classification/dev_pairs.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        dev_sentences1.append(row['product1'])
        dev_sentences2.append(row['product2'])
        dev_labels.append(int(row['is_duplicate']))


binary_acc_evaluator = evaluation.BinaryClassificationEvaluator(dev_sentences1, 
                                                                dev_sentences2, 
                                                                dev_labels)
evaluators.append(binary_acc_evaluator)

In [15]:
###### Duplicate Product Mining ######
# Given a large corpus of products, identify all duplicates in that corpus.

# For faster processing, we limit the development corpus to only 10,000 sentences.
max_dev_samples = 100_000
dev_sentences = {}
dev_duplicates = []
with open(os.path.join(dataset_path, "duplicate-mining/dev_corpus.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        dev_sentences[row['qid']] = row['product']

        if len(dev_sentences) >= max_dev_samples:
            break

with open(os.path.join(dataset_path, "duplicate-mining/dev_duplicates.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row['qid1'] in dev_sentences and row['qid2'] in dev_sentences:
            dev_duplicates.append([row['qid1'], row['qid2']])

# The ParaphraseMiningEvaluator computes the cosine similarity between all sentences and
# extracts a list with the pairs that have the highest similarity. Given the duplicate
# information in dev_duplicates, it then computes and F1 score how well our duplicate mining worked
paraphrase_mining_evaluator = evaluation.ParaphraseMiningEvaluator(dev_sentences, dev_duplicates, name='dev')
evaluators.append(paraphrase_mining_evaluator)

In [18]:
###### Duplicate Products Information Retrieval ######
# Given a product and a large corpus of thousands products, 
# find the most relevant (i.e. duplicate) product in that corpus.

# For faster processing, we limit the development corpus to only 10,000 sentences.
max_corpus_size = 100_000

ir_queries = {}             #Our queries (qid => product)
ir_needed_qids = set()      #QIDs we need in the corpus
ir_corpus = {}              #Our corpus (qid => product)
ir_relevant_docs = {}       #Mapping of relevant documents for a given query 
                            #(qid => set([relevant_product_ids])

with open(os.path.join(dataset_path, 'information-retrieval/dev-queries.tsv'), encoding='utf8') as fIn:
    next(fIn) #Skip header
    for line in fIn:
        qid, query, duplicate_ids = line.strip().split('\t')
        duplicate_ids = duplicate_ids.split(',')
        ir_queries[qid] = query
        ir_relevant_docs[qid] = set(duplicate_ids)

        for qid in duplicate_ids:
            ir_needed_qids.add(qid)

In [23]:
# First get all needed relevant documents 
# (i.e., we must ensure, that the relevant products are actually in the corpus
distraction_products = {}
with open(os.path.join(dataset_path, 'information-retrieval/corpus.tsv'), encoding='utf8') as fIn:
    next(fIn) #Skip header
    for line in fIn:
        qid, product = line.strip().split('\t')

        if qid in ir_needed_qids:
            ir_corpus[qid] = product
        else:
            distraction_products[qid] = product

# Now, also add some irrelevant products to fill our corpus
other_qid_list = list(distraction_products.keys())
random.shuffle(other_qid_list)

for qid in other_qid_list[0:max(0, max_corpus_size-len(ir_corpus))]:
    ir_corpus[qid] = distraction_products[qid]

In [26]:
#Given queries, a corpus and a mapping with relevant documents, 
# the InformationRetrievalEvaluator computes different IR metrices. 
# For our use case MRR@k and Accuracy@k are relevant.
ir_evaluator = evaluation.InformationRetrievalEvaluator(ir_queries, ir_corpus, ir_relevant_docs)

evaluators.append(ir_evaluator)

In [27]:
# Create a SequentialEvaluator. This SequentialEvaluator runs all three evaluators in a sequential order.
# We optimize the model with respect to the score from the last evaluator (scores[-1])
seq_evaluator = evaluation.SequentialEvaluator(evaluators, main_score_function=lambda scores: scores[-1])


logger.info("Evaluate model without training")
with torch.no_grad():
    seq_evaluator(model, epoch=0, steps=0, output_path=model_save_path)

2022-11-16 10:24:43 - Evaluate model without training
2022-11-16 10:24:43 - Binary Accuracy Evaluation of the model on  dataset in epoch 0 after 0 steps:
2022-11-16 10:24:57 - Accuracy with Cosine-Similarity:           79.27	(Threshold: 0.8767)
2022-11-16 10:24:57 - F1 with Cosine-Similarity:                 55.65	(Threshold: 0.7286)
2022-11-16 10:24:57 - Precision with Cosine-Similarity:          46.94
2022-11-16 10:24:57 - Recall with Cosine-Similarity:             68.32
2022-11-16 10:24:57 - Average Precision with Cosine-Similarity:  47.39

2022-11-16 10:24:57 - Accuracy with Manhattan-Distance:           79.27	(Threshold: 7.6162)
2022-11-16 10:24:57 - F1 with Manhattan-Distance:                 56.22	(Threshold: 11.4881)
2022-11-16 10:24:57 - Precision with Manhattan-Distance:          47.30
2022-11-16 10:24:57 - Recall with Manhattan-Distance:             69.31
2022-11-16 10:24:57 - Average Precision with Manhattan-Distance:  47.65

2022-11-16 10:24:57 - Accuracy with Euclidean-Di

In [28]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=seq_evaluator,
          epochs=num_epochs,
          warmup_steps=1000,
          output_path=model_save_path
          )

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/26 [00:00<?, ?it/s]

2022-11-16 10:26:31 - Binary Accuracy Evaluation of the model on  dataset after epoch 0:
2022-11-16 10:26:32 - Accuracy with Cosine-Similarity:           79.27	(Threshold: 0.8836)
2022-11-16 10:26:32 - F1 with Cosine-Similarity:                 55.65	(Threshold: 0.7315)
2022-11-16 10:26:32 - Precision with Cosine-Similarity:          46.94
2022-11-16 10:26:32 - Recall with Cosine-Similarity:             68.32
2022-11-16 10:26:32 - Average Precision with Cosine-Similarity:  47.59

2022-11-16 10:26:32 - Accuracy with Manhattan-Distance:           79.50	(Threshold: 7.6096)
2022-11-16 10:26:32 - F1 with Manhattan-Distance:                 56.22	(Threshold: 11.4671)
2022-11-16 10:26:32 - Precision with Manhattan-Distance:          47.30
2022-11-16 10:26:32 - Recall with Manhattan-Distance:             69.31
2022-11-16 10:26:32 - Average Precision with Manhattan-Distance:  47.74

2022-11-16 10:26:32 - Accuracy with Euclidean-Distance:           79.27	(Threshold: 0.4824)
2022-11-16 10:26:32 -

Iteration:   0%|          | 0/26 [00:00<?, ?it/s]

2022-11-16 10:26:40 - Binary Accuracy Evaluation of the model on  dataset after epoch 1:
2022-11-16 10:26:40 - Accuracy with Cosine-Similarity:           79.27	(Threshold: 0.8883)
2022-11-16 10:26:40 - F1 with Cosine-Similarity:                 55.46	(Threshold: 0.7481)
2022-11-16 10:26:40 - Precision with Cosine-Similarity:          48.18
2022-11-16 10:26:40 - Recall with Cosine-Similarity:             65.35
2022-11-16 10:26:40 - Average Precision with Cosine-Similarity:  47.78

2022-11-16 10:26:40 - Accuracy with Manhattan-Distance:           79.27	(Threshold: 7.3747)
2022-11-16 10:26:40 - F1 with Manhattan-Distance:                 56.33	(Threshold: 11.2833)
2022-11-16 10:26:40 - Precision with Manhattan-Distance:          47.92
2022-11-16 10:26:40 - Recall with Manhattan-Distance:             68.32
2022-11-16 10:26:40 - Average Precision with Manhattan-Distance:  47.89

2022-11-16 10:26:40 - Accuracy with Euclidean-Distance:           79.27	(Threshold: 0.4726)
2022-11-16 10:26:40 -

...

2022-11-16 10:26:49 - Binary Accuracy Evaluation of the model on  dataset after epoch 2:
2022-11-16 10:26:49 - Accuracy with Cosine-Similarity:           79.27	(Threshold: 0.8907)
2022-11-16 10:26:49 - F1 with Cosine-Similarity:                 55.52	(Threshold: 0.6654)
2022-11-16 10:26:49 - Precision with Cosine-Similarity:          40.74
2022-11-16 10:26:49 - Recall with Cosine-Similarity:             87.13
2022-11-16 10:26:49 - Average Precision with Cosine-Similarity:  48.33

2022-11-16 10:26:49 - Accuracy with Manhattan-Distance:           79.27	(Threshold: 7.2763)
2022-11-16 10:26:49 - F1 with Manhattan-Distance:                 55.83	(Threshold: 11.9874)
2022-11-16 10:26:49 - Precision with Manhattan-Distance:          43.41
2022-11-16 10:26:49 - Recall with Manhattan-Distance:             78.22
2022-11-16 10:26:49 - Average Precision with Manhattan-Distance:  48.70

2022-11-16 10:26:49 - Accuracy with Euclidean-Distance:           79.27	(Threshold: 0.4675)
2022-11-16 10:26:49 -

Iteration:   0%|          | 0/26 [00:00<?, ?it/s]

2022-11-16 10:26:57 - Binary Accuracy Evaluation of the model on  dataset after epoch 3:
2022-11-16 10:26:57 - Accuracy with Cosine-Similarity:           79.50	(Threshold: 0.8942)
2022-11-16 10:26:57 - F1 with Cosine-Similarity:                 56.85	(Threshold: 0.7002)
2022-11-16 10:26:57 - Precision with Cosine-Similarity:          43.46
2022-11-16 10:26:57 - Recall with Cosine-Similarity:             82.18
2022-11-16 10:26:57 - Average Precision with Cosine-Similarity:  49.11

2022-11-16 10:26:57 - Accuracy with Manhattan-Distance:           79.50	(Threshold: 6.6169)
2022-11-16 10:26:57 - F1 with Manhattan-Distance:                 57.35	(Threshold: 11.8518)
2022-11-16 10:26:57 - Precision with Manhattan-Distance:          44.94
2022-11-16 10:26:57 - Recall with Manhattan-Distance:             79.21
2022-11-16 10:26:57 - Average Precision with Manhattan-Distance:  49.43

2022-11-16 10:26:57 - Accuracy with Euclidean-Distance:           79.50	(Threshold: 0.4598)
2022-11-16 10:26:57 -

Iteration:   0%|          | 0/26 [00:00<?, ?it/s]

2022-11-16 10:27:06 - Binary Accuracy Evaluation of the model on  dataset after epoch 4:
2022-11-16 10:27:06 - Accuracy with Cosine-Similarity:           79.50	(Threshold: 0.9002)
2022-11-16 10:27:06 - F1 with Cosine-Similarity:                 56.87	(Threshold: 0.6769)
2022-11-16 10:27:06 - Precision with Cosine-Similarity:          41.98
2022-11-16 10:27:06 - Recall with Cosine-Similarity:             88.12
2022-11-16 10:27:06 - Average Precision with Cosine-Similarity:  49.59

2022-11-16 10:27:06 - Accuracy with Manhattan-Distance:           79.73	(Threshold: 8.1318)
2022-11-16 10:27:06 - F1 with Manhattan-Distance:                 57.97	(Threshold: 11.7009)
2022-11-16 10:27:06 - Precision with Manhattan-Distance:          45.71
2022-11-16 10:27:06 - Recall with Manhattan-Distance:             79.21
2022-11-16 10:27:06 - Average Precision with Manhattan-Distance:  49.95

2022-11-16 10:27:06 - Accuracy with Euclidean-Distance:           79.50	(Threshold: 0.4467)
2022-11-16 10:27:06 -